In [1]:
import os
from train_members import train_model
from utils.config import *
from data import load_data
from models import models
import transformation
import random
import pandas as pd

/home/students/student4_6/anaconda3/envs/athena/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/students/student4_6/anaconda3/envs/athena/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/students/student4_6/anaconda3/envs/athena/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/s

PROJECT DICTIONARY: /home/students/student4_6/athena-master/src


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
finished = [tmp.split('.')[0] for tmp in os.listdir('data/results_new')]
model_names = [tmp for tmp in os.listdir('data/models') if tmp.split('.')[0] not in finished]
len(model_names)

0

In [3]:
_, mnist_test = load_data('mnist')
y = mnist_test[1].argmax(axis=-1)

Dataset(MNIST) Summary:
Train set: (60000, 28, 28, 1), (60000, 10)
Test set: (10000, 28, 28, 1), (10000, 10)


In [4]:
# for model in model_names:
#     cur_model = models.load_model(model)
#     pred_y = cur_model.predict(mnist_test[0])
#     accu = (pred_y.argmax(axis=1) == mnist_test[1].argmax(axis=1)).astype(np.int).sum() / mnist_test[0].shape[0]
#     print("{} accuracy: {:6f}".format(model.split('.')[0], accu))
#     np.save(os.path.join('data/results_new/', model.split('.')[0]), pred_y)
#     del cur_model, pred_y
# save predictions

In [5]:
# if not os.path.exists(os.path.join('data/results_new', 'predictions_all_new.npy')):
#     pred = np.empty((len(TRANSFORMATION.supported_types()), 10000, 10))
#     npfiles = [tmp for tmp in os.listdir('data/results_new') if '.npy' in tmp]
#     i = 0
#     for npfile in npfiles:
#         tmp = np.load(os.path.join('data/results_new', npfile), allow_pickle=True)
#         if (np.isnan(tmp).astype(np.int)).sum() > 0:
#             print(npfile)
#         else:
#             pred[i] = tmp
#         i += 1
#     np.save(os.path.join('data/results_new', 'predictions_all_new'), pred)
# else:
#     print('loading......')
#     pred = np.load(os.path.join('data/results_new', 'predictions_all_new.npy'), allow_pickle=True)

In [6]:
def random_defense(preds):
    return preds[random.randint(0,72)].argmax(axis=-1)

def majority_vote(preds):
    tmp = preds.argmax(axis=-1)
    out = np.empty(tmp.shape[-1])
    for i in range(tmp.shape[-1]):
        out[i] = np.bincount(tmp[:,i]).argmax()
    return out

def top_2_majority_vote(preds):
    tmp = preds.argsort(axis=-1)[:, :, -2:]
    out = np.empty(preds.shape[1])
    for i in range(out.shape[0]):
        out[i] = np.bincount(tmp[:, i, :].reshape(-1)).argmax()
    return out

def average_probability(preds):
    return preds.sum(axis=0).argmax(-1)

# Craft and evaluate on adversarial examples generated using ART

In [7]:
import art
x = mnist_test[0]

In [8]:
model_name = 'model-mnist-cnn-clean.h5'

In [9]:
emsemble_strat = {
    'RD': random_defense,
    'MV': majority_vote,
    'T2MV': top_2_majority_vote,
    'AP': average_probability
}

## evaluate on adversarial examples

In [10]:
model_names = os.listdir('data/models')
adv_examples = [tmp for tmp in os.listdir('data/adv_examples/') if '.npy' in tmp]
result = pd.DataFrame(columns=list(emsemble_strat.keys()), index=[tmp.split('.')[0] for tmp in adv_examples])

In [ ]:
for adv_name in adv_examples:
    # create directory to store result
    dir_name = os.path.join('data', 'adv_results', adv_name.split('.')[0])
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    adv_x = np.load(os.path.join('data/adv_examples', adv_name))
    preds = np.empty((73, 10000, 10))
    cnt = 0
    for model_name in model_names:
        if not os.path.exists(os.path.join(dir_name, model_name.split('.')[0] + '.npy')):
            model = models.load_model(model_name)
            tmp_out = model.predict(adv_x)
            np.save(os.path.join(dir_name, model_name.split('.')[0]), tmp_out)
        else:
            print('loading')
            tmp_out = np.load(os.path.join(dir_name, model_name.split('.')[0] + '.npy'), allow_pickle=True)
        preds[cnt] = tmp_out
    out = {}
    for ensemble in result.columns:
        out[ensemble] = ((emsemble_strat[ensemble](preds) == y).astype(np.int).sum() / y.shape).item()
    result.loc[adv_name.split('.')[0]] = out
    print("Done evaluating {}".format(adv_name.split('.')[0]))

loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
Done evaluating bim_ord2_nbIter100_eps750
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loading
loadin